In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential, Model
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, Embedding,Dropout, Bidirectional, Input, merge
from keras.preprocessing.sequence import pad_sequences
import keras
import json
from data_utils import *
from model import *

C:\Users\Gzhang\AppData\Local\Continuum\Anaconda2\envs\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
SENTENCE_LENGTH = 20
VOCAB_SIZE = 10000

def load_cornell_movie_data(start_line, limit, vocab_dict):
    sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, start_line, limit)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_twitter_data(start_line, limit, vocab_dict):
    sentences = read_sentences("ShortenTwitterAsciiCorpus.txt")[1000:2000]
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_friends_data():
    sentences = read_lines("friends-final.txt", "	", 2, 5, start_line = 0, limit = 10000)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

In [16]:
#model = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 100, 3)
model = create_model_with_attention(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 300, 100, 3)
print(model.summary())
model.load_weights("0new_model.h5")

C:\Users\Gzhang\Project\MLPractice\chatbot\model.py:29: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  output_att = merge([inputs, att_prob ], mode='mul')
C:\Users\Gzhang\AppData\Local\Continuum\Anaconda2\envs\python36\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 300)      3000000     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 20, 200)      240600      embedding_2[0][0]                
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 20, 200)      40200       bidirectional_2[0][0]            
__________________________________________________________________________________________________
merge_2 (M

In [4]:
vocab_dict, idx_dict = load_vocab("vocab_dict.json")

In [17]:
batch_size = 1000
start_point = 8000
batch = 8
round = 0
while True:
    print("Getting data:", start_point, batch_size)
    x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_dict)
    y = np.argmax(y_vec, axis=2)
    print(" ".join([idx_dict[x] for x in y[0] ]) )
    
    if len(x_vec) == 0:
        start_point = 0
        batch = 0
        x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_dict)
        round += 1 
    model.fit(x_vec, y_vec, batch_size=100, nb_epoch=100, shuffle = True)
    model.save_weights(str(batch%2)+"new_model.h5")
    start_point += batch_size
    batch += 1
    print("--------------------------------------------------------------------------")
    print("Round:", round, "Batch:", batch)
    print("--------------------------------------------------------------------------")

Getting data: 8000 1000
(96, 20, 10000)
                                      hi


C:\Users\Gzhang\AppData\Local\Continuum\Anaconda2\envs\python36\lib\site-packages\ipykernel_launcher.py:16: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  app.launch_new_instance()


Epoch 1/100
96/96 [==============================] - 4s 41ms/step - loss: 10.1013 - acc: 0.3333
Epoch 2/100
96/96 [==============================] - 2s 20ms/step - loss: 8.7011 - acc: 0.3333
Epoch 3/100
96/96 [==============================] - 2s 20ms/step - loss: 8.0461 - acc: 0.3333
Epoch 4/100
96/96 [==============================] - 2s 20ms/step - loss: 6.7653 - acc: 0.3333
Epoch 5/100
96/96 [==============================] - 2s 21ms/step - loss: 5.5599 - acc: 0.6667
Epoch 6/100
96/96 [==============================] - 2s 21ms/step - loss: 5.4388 - acc: 0.6667
Epoch 7/100
96/96 [==============================] - 2s 20ms/step - loss: 5.4170 - acc: 0.6667
Epoch 8/100
96/96 [==============================] - 2s 22ms/step - loss: 5.4055 - acc: 0.6667
Epoch 9/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3978 - acc: 0.6667
Epoch 10/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3922 - acc: 0.6667
Epoch 11/100
96/96 [============================

Epoch 87/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 88/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 89/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 90/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 91/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 92/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 93/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 94/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 95/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 96/100
96/96 [==============================] - 2s 20ms/step - loss: 5.3727 - acc: 0.6667
Epoch 97/100
96/96 [====================

KeyboardInterrupt: 

In [21]:
#do you clean
sen = "hi"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '').split(" ")
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)
print(res)
vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0] ])
#if idx_dict[x]!= 'UNK'

[[[8.5367309e-04 1.5468397e-03 1.1177040e-03 ... 7.7047422e-05
   7.6753699e-05 1.3228448e-03]
  [8.5367309e-04 1.5468397e-03 1.1177040e-03 ... 7.7047422e-05
   7.6753699e-05 1.3228448e-03]
  [8.5367309e-04 1.5468397e-03 1.1177040e-03 ... 7.7047422e-05
   7.6753699e-05 1.3228448e-03]
  ...
  [8.5367309e-04 1.5468397e-03 1.1177040e-03 ... 7.7047422e-05
   7.6753699e-05 1.3228448e-03]
  [8.5367309e-04 1.5468397e-03 1.1177040e-03 ... 7.7047422e-05
   7.6753699e-05 1.3228448e-03]
  [4.6896185e-03 8.1577174e-02 9.0191752e-04 ... 6.1995437e-10
   6.3818367e-10 8.9655733e-01]]]


'you you you you you you you you you you you you you you you you you you you UNK'